In [1]:
import json
import os
import random
import time

In [2]:
import h3
import geopandas as gpd
import pandas as pd

In [3]:
from kroger_api import KrogerAPI
from kroger_api.utils.env import load_and_validate_env, get_zip_code
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [6]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [7]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [8]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Store Data

In [9]:
load_and_validate_env(["KROGER_CLIENT_ID", "KROGER_CLIENT_SECRET"])
zip_code = get_zip_code(default="10001")

In [10]:
kroger = KrogerAPI()
token_info = kroger.authorization.get_token_with_client_credentials("product.compact")

Found saved token, will test if it's still valid...
Token appears invalid, requesting a new one


In [11]:
res = 3
distance = 0.5

usa_geom = states_df[~states_df["STUSPS"].isin(["PR", "VI", "GU", "MP", "AS"])][
    "geometry"
].union_all()

h3_shape = h3.geo_to_h3shape(usa_geom.buffer(distance=distance))
h3_cells = h3.h3shape_to_cells(h3_shape, res=res)
geojson = h3.cells_to_geo(h3_cells)

In [12]:
stores = []
for cell in tqdm(h3_cells, desc="Parsing Stores"):
    lat, lng = h3.cell_to_latlng(cell)
    locations = kroger.location.search_locations(
        lat=lat, lon=lng, radius_in_miles=100, limit=200
    )
    if len(locations["data"]):
        for location in locations["data"]:
            location["geometry"] = Point(
                location["geolocation"]["longitude"],
                location["geolocation"]["latitude"],
            )
            location["STUSPS"] = location["address"]["state"]
            stores.append(location)
    else:
        continue
    time.sleep(random.uniform(0.01, 0.5))

Parsing Stores:   0%|          | 0/911 [00:00<?, ?it/s]

In [45]:
stores_gdf = gpd.GeoDataFrame(stores, crs=4326)
stores_gdf = stores_gdf.drop_duplicates("locationId")
stores_gdf.to_file("data/all_stores.gpkg")

In [17]:
stores_gdf = gpd.read_file("data/all_stores.gpkg")

In [18]:
rulers_gdf = stores_gdf[stores_gdf["chain"] == "RULER"]
rulers_gdf = rulers_gdf[~rulers_gdf["departments"].isna()].reset_index(drop=True)

In [19]:
rulers_gdf = rulers_gdf.to_crs(3435)
rulers_gdf.to_file("data/stores.gpkg")

In [20]:
rulers_count_df = rulers_gdf.groupby("STUSPS").size().reset_index(name="COUNT")

In [21]:
rulers_count_df

,STUSPS,COUNT
0,IL,11
1,IN,15
2,KY,7
3,MO,6
4,OH,4
5,TN,2


## Combine With States

In [23]:
store_count_gdf = states_with_population_df.merge(
    rulers_count_df, on="STUSPS", how="left"
)

In [24]:
store_count_gdf = store_count_gdf.dropna()

In [25]:
store_count_gdf["per_100k"] = (
    store_count_gdf["COUNT"] / (store_count_gdf["POPULATION"] / 100_000)
).round(decimals=2)
store_count_gdf["per_1m"] = (
    store_count_gdf["COUNT"] / (store_count_gdf["POPULATION"] / 1_000_000)
).round(decimals=2)

In [26]:
store_count_gdf = store_count_gdf.to_crs(3435)
store_count_gdf.to_file(f"data/Ruler_Foods_Per_State.gpkg")

In [27]:
store_count_gdf.sort_values("per_1m", ascending=False)

,STUSPS,NAME,POPULATION,geometry,COUNT,per_100k,per_1m
53,IN,Indiana,6880131.0,"POLYGON ((1052220.397 450617.848, 1053166.131 ...",15.0,0.22,2.18
47,KY,Kentucky,4550595.0,"MULTIPOLYGON (((669175.722 -48668.604, 671280....",7.0,0.15,1.54
20,MO,Missouri,6208038.0,"POLYGON ((-1083377.049 1512139.592, -1081822.5...",6.0,0.10,0.97
29,IL,Illinois,12642259.0,"POLYGON ((95734.593 1295783.036, 96366.831 129...",11.0,0.09,0.87
48,OH,Ohio,11824034.0,"MULTIPOLYGON (((2515458.157 1847805.749, 25159...",4.0,0.03,0.34
23,TN,Tennessee,7148304.0,"POLYGON ((392151.805 -599273.721, 392317.228 -...",2.0,0.03,0.28
